# 10. Git Server

다룰 내용: 프로토콜, 서버 구성, 서버 관리

* `<   >`은 괄호 내부가 다른 항목으로 대체될 수 있음

## 1. 프로토콜

* Git 서버를 운영하기 위해서는 먼저 사용할 전송 프로토콜을 정해야 한다.
* 프로토콜을 사용하는 방법, 사용할 수 있도록 서버를 구성하는 방법을 알아야 한다.
![protocol](fig 10/fig10-1.png)


### 1-1. 로컬 프로토콜

* 리모트 저장소가 같은 시스템의 다른 디렉토리에 있는 경우에 사용한다.
![local protocol](fig 10/fig10-2.png)
![local protocol](fig 10/fig10-3.png)
* 팀원들이 전부 하나의 시스템에서 작업하게 되므로 저장소가 손상되면 복구할 수 없다.
* 시스템에서 로컬 저장소를 사용하는 것처럼 `Clone - Push - Pull` 한다.
* 저장소를 `Clone` 하거나 프로젝트에 리모트 저장소로 추가한다.
* `git clone </srv/git/project.git>` : 필요한 파일을 직접 복사하거나 하드 링크를 사용한다. 속도가 빠르다.
* `git clone file://</srv/git/project.git>` : 프로세스를 별도로 생성하여 처리한다. 효율은 떨어지지만 저장소의 복사본을 깨끗한 상태로 남겨준다. (외부 Refs나 개체들)
* `git remote add <local_proj> </srv/git/project.git>` : 이미 존재하는 프로젝트에 로컬 저장소를 추가한다. 네트워크에 있는 리모트 저장소처럼 `local_proj`에 Push, Pull 할 수 있다.

* 장점
  * 기존에 있던 네트워크, 권한을 그대로 사용하기 때문에 쉽고 간단하다.
  * Bare 저장소를 만들어 시스템 내에 모든 사용자가 읽고 쓸 수 있다.
  * A 사용자가 서버에 Push한 뒤에 B 사용자가 Pull 할 필요 없이 `git pull </home/john/project>` 로 B 사용자는 A 사용자의 파일을 확인할 수 있다.

* 단점
  * 한 시스템을 사용하기 때문에 다양한 상황에서 공유 디렉토리에 접근하는 것이 어렵다.
  * 예를 들어 서버가 있는 곳이 아닌 집에서 Push 해야 할 때
  * 저장소에 문제가 생겨도 보호하기가 쉽지 않다. 보안에 취약하다.


### 1-2. HTTP(hypertext transfer protocol)

* 인터넷에서 Hypertext 문서를 교환하기 위하여 사용되는 통신규약이다.
* Hypertext: 사용자가 연상하는 순서에 따라 원하는 정보를 얻을 수 있는 시스템 <-> 계속 일정한 정보를 순차적으로 얻음(일반 텍스트)
![http protocol](fig 10/fig10-4.png)

* Smart HTTP
  * SSH(키를 발급하고 관리), Git(익명) 프로토콜처럼 통신한다.
  * HTTP, HTTPS 포트를 이용해 통신하고 다양한 HTTP 인증 방식을 사용한다.
  * 사용자이름과 암호만으로 인증할 수 있기 때문에 더 편리하게 사용할 수 있다.
  * Git에서 가장 많이 사용하는 프로토콜이다.

* Dumb HTTP
  * Git 서버가 Smart HTTP 요청에 응답하지 않으면 Dumb HTTP를 시도한다.
  * 원격 저장소를 그냥 파일 건네주는 웹 서버로 취급한다.
  * 보통 Smart HTTP, Dumb HTTP 중 하나만 사용한다.
  
* 장점
  * 읽기와 쓰기에 하나의 URL만 사용한다.
  * 사용자에게 익숙한 사용자이름과 암호 방식의 인증을 사용하여 SSH보다 간단하다.
  * 보편적이며 빠르고 효율적이다.

* 단점
  * Push 할 때 SSH 보다 좀 더 복잡하다.
  
### 1-3. SSH(Secure Shell) 프로토콜

* 공개 키 암호 방식을 사용하여 원격 시스템에 접근하여 암호화된 메시지를 전송할 수 있는 시스템이다.
![ssh protocol](fig 10/fig10-5.png)
* 원격접속을 안전하게 할 수 있게 해준다.
* SSH를 이용하면 아무런 외부 도구 없이 Git 서버를 구축할 수 있다.
* SSH는 인증 기능이 있고 어디에서든 쉽게 사용할 수 있다.
* `git clone ssh://[user@]server/project.git` SSH를 통해 Git 저장소를 Clone할 수 있다.

* 장점
  * 비교적 설정하기 쉽고 SSH 데몬은 흔하다.
  * 데몬(daemon): 사용자가 직접적으로 제어하지 않고, 백그라운드에서 돌면서 여러 작업을 하는 프로그램
  * 모든 데이터는 암호화되어 인증된 상태로 전송되므로 보안이 철저하다.
  * 데이터를 압축하므로 효율적이다.(HTTPS, Local 프로토콜과 마찬가지)
  
* 단점
  * 익명으로 접근할 수 없다.
  * 회사에서 사용할 땐 적합하지만 오픈소스 프로젝트에선 부족하다.
  * 익명으로 접근하려면 Push만 SSH가 가능하고, Fetch는 다른 프로토콜을 이용해야 한다.
  
### 1-4. Git 프로토콜

* Git에 포함된 데몬을 사용하는 것이다.
* 포트는 9418, SSH와 비슷하지만 인증 메커니즘이 없다.
* 저장소에 `git-export-daemon-ok` 파일을 만들면 Git 프로토콜로 서비스할 수 있다.
* 누구나 Clone 함 vs 아무도 Clone 할 수 없음

* 장점
  * 전송속도가 가장 빠르다.
  * 전송량이 많은 공개 프로젝트나 별도의 인증이 필요 없고 읽기만 허용하는 프로젝트를 서비스할 때 유용하다.
  * `Git 프로토콜 + 암호화 + 인증` = `SSH 프로토콜`
  
* 단점
  * 인증 메커니즘이 없다. SSH나 HTTPS와 함께 사용한다.
  * 가장 설치하기 어려운 방법이다.
  * 별도의 데몬이 필요하고 프로젝트에 맞게 설정해야 한다.
  * 방화벽을 통과할 수 있도록 9418 포트도 열어야 한다.
  
### 1-5. 결론

* 랩실에서 사용하기 위한 프로토콜은 HTTP 혹은 SSH가 가장 적합할 것이다.

## 2. 서버에 Git 설치하기

* 서버에 Git을 설치하고 설정하려면 보안 설정과 OS 도구를 사용해야 한다.
* Linux 기준 설명이다. (현재 랩실 서버 컴퓨터에 적용)

### 2-1. Bare 저장소 만들기

* 서버에 설치하기 위해 저장소를 Bare 저장소(워킹 디렉토리가 없는 저장소)로 만들어야 한다.
![working directory](fig 10/fig10-6.png)
* `--bare` 옵션을 주고 Clone 하면 새로운 Bare 저장소가 만들어진다. 디렉토리는 관례에 따라 `.git` 가 붙는다.
* `git clone --bare my_project my_project.git` : `my_project.git` 디렉토리에는 복사한 Git 디렉토리 데이터만 들어 있다.

### 2-2. 서버에 Bare 저장소 넣기

* Bare 저장소를 서버에 넣고 프로토콜을 설정한다.
* 서버 `git.example.com`를 준비하고 SSH로 접속할 수 있게 만든다.
* `scp -r my_project.git user@git.example.com:/srv/git` : Bare 저장소를 `/srv/git` 디렉토리에 복사한다.
* `git clone user@git.example.com:/srv/git/my_project.git` : 다른 사용자들은 SSH로 서버에 접근해서 저장소를 Clone 할 수 있다. 사용자는 `/srv/git` 디렉토리에 읽기 권한이 있어야 한다.
* 디렉토리에 쓰기 권한이 있는 경우 Push도 할 수 있다.
* `git init --bare --shared` : `git init` 명령에 `--shared` 옵션을 추가하면 그룹 쓰기 권한을 추가한다.

* SSH로 접속할 수 있도록 서버에 계정을 만들고 Bare 저장소를 사람들이 읽고 쓸 수 있는 곳에 넣어 두기만 하면 된다.

### 2-3. SSH 접속

* 모든 개발자가 SSH로 접속할 수 있는 서버가 있으면 매우 쉽게 저장소를 만들 수 있다.
* 사용자들에게 접속 권한을 주기 위해 모두에게 계정을 만들어 준다.
  * 제일 단순하지만 다소 귀찮을 수 있다.
  * 사용자마다 `adduser`를 실행시키고 임시 암호를 부여해야 한다.
  * 다른 방법도 있지만 현재 랩실 규모에서 사용하기 좋은 방법이다.
  
### 2-4. SSH 공개키 만들기

* 공개키를 만들기 위해 키가 있는지 확인해야 한다.
* 사용자의 SSH 키들은 기본적으로 사용자의 `~/.ssh` 디렉토리에 저장한다.
* `cd ~/.ssh` 이후 `ls` (ls는 파일 목록을 보여준다.) : 디렉토리의 파일을 살펴본다.
  * `id_dsa` 혹은 `id_rsa` 라는 파일과 `.pub` 확장자가 붙은 파일이 있을 것이다. `.pub`는 공개키이고 나머지는 개인키이다.
  * 만약 이 파일들이 없거나 `.ssh` 디렉토리가 없는 경우 `ssh-keygen` 프로그램을 이용해 키를 생성해야 한다.
  * `ssh-keygen`은 Linux나 Mac의 SSH 패키지에 포함돼 있고 Windows는 Git for Windows 안에 들어 있다.
  * SSH 키에 관한 매뉴얼 https://help.github.com/en/articles/connecting-to-github-with-ssh
  

# 3. Git 서버 설정하기

* Git 계정을 만들고 사용자 홈 디렉토리에 `.ssh` 디렉토리를 만든다.
  * ` sudo adduser git ` # git 계정 생성
  * ` su git ` # 생성한 계정으로 전환
  * ` cd  ` # 홈 디렉토리(/home/git)로 이동
  * ` mkdir .ssh && chmod 700 .ssh  ` # 디렉토리 생성 && .ssh 디렉토리 권한 설정
  * ` touch .ssh/authorized_keys && chmod 600 .ssh/authorized_keys ` # 기존에 존재하는 파일 수정 && authorized_keys 파일 권한 설정

* `authorized_keys` 파일에 다른 사용자의 SSH 공개키를 추가한다.
  * `cat /tmp/id_rsa.john.pub >> ~/.ssh/authorized_keys`

* `--bare` 옵션에 `git init`를 실행하여 워킹 디렉토리가 없는 빈 저장소를 만든다.
  * `cd /srv/git` # git 저장소 디렉토리
  * `mkdir project.git` # 디렉토리 생성
  * `cd project.git` # 디렉토리 경로로 이동
  * `git init --bare` # bare 옵션으로 .git 하위 디렉토리 생성
  * 등록된 사용자는 이 저장소를 리모트 저장소로 등록한 뒤 브랜치를 Push 할 수 있다.
  * 프로젝트마다 적어도 한 명은 서버에 접속해서 Bare 저장소를 만들어야 한다.

* 프로젝트를 Clone 하고 나서 수정하고 Push 하도록 설정한다.
  * `cd myproject` # 이미 myproject가 있는 경우
  * `git init`
  * `git add .` # git ignore와 `git add *` (전체 파일) 사이의 충돌을 막음
  * `git commit -m 'initial commit'` # 커밋하기 (커밋이름)
  * `git remote add origin git@gitserver:/srv/git/project.git` # 워킹디렉토리에 새 리모트 저장소 추가
  * `git push origin master` # origin에 master를 push함
  
* Git 계정으로 서버에 로그인할 수 있다.
  * 이를 막으려면 로그인셸을 `git-shell`로 수정한다.
  * `cat /etc/shells` # 이미 `git-shell` 이 등록돼 있는지 확인
  * `which git-shell` # git-shell 실행파일이 설치돼 있는지 확인
  * `sudo vim /etc/shells` # 바로 위 명령으로 확인한 git-shell 실행파일의 절대경로를 추가
  * `chsh <계정이름> -s <shell>` 명령어를 이용해서 특정 계정의 셸을 바꿀 수 있다.
  * `sudo chsh git` # git-shell 경로를 입력, 보통 /usr/bin/git-shell
  * Git은 제대로 동작하되 개발자들이 셸을 얻지 못한다.

참고

* https://git-scm.com/book/ko/v2/Git-%EC%84%9C%EB%B2%84-%ED%94%84%EB%A1%9C%ED%86%A0%EC%BD%9C progit
* http://blogs.innovationm.com/http-protocol/ HTTP protocol
* https://terms.naver.com/entry.nhn?docId=1180001&cid=40942&categoryId=32851 HTTP
* https://www.ssh.com/ssh/protocol/ ssh
* https://terms.naver.com/entry.nhn?docId=837917&cid=50376&categoryId=50376 SSH
* https://terms.naver.com/entry.nhn?docId=3351634&cid=40942&categoryId=32828 SSH
* https://ko.wikipedia.org/wiki/%EB%8D%B0%EB%AA%AC_(%EC%BB%B4%ED%93%A8%ED%8C%85) Daemon
* http://wanochoi.com/?p=1551 su git
* https://mediatemple.net/community/products/dv/204643550/common-ssh-commands touch
* https://sanghaklee.tistory.com/31 git add .